In [ ]:
interface MyP {
  then(onResolve: Function, onReject: Function): MyP
  catch(onReject: Function): MyP
  finally(onFinally: Function): MyP
}

enum PStatus {
  'pending',
  'resolved',
  'rejected'
}

class MyP {
  private status: PStatus
  private executor: Function
  private res: any
  private cbArr: Array<any> = []
  private errcbArr: Array<any> = []
  private fincbArr: Array<any> = []

  constructor(executor) {
    this.status = PStatus.pending
    this.executor = executor
    this.res = null

    try {
      executor(this.onResolve.bind(this), this.onReject.bind(this))
    } catch (err) {
      this.onReject(err)
    }
  }

  private run() {
    if (this.status === PStatus.rejected) {
      for (var i = 0; i < this.errcbArr.length; i++) {
        this.errcbArr[i](this.res)
      }
      return
    }

    try {
      for (var i = 0; i < this.cbArr.length; i++) {
        this.cbArr[i](this.res)
      }
    }
    catch {
      for (var i = 0; i < this.errcbArr.length; i++) {
        this.errcbArr[i](this.res)
      }
    }
  }

  onResolve(res: any) {
    if (this.status === PStatus.pending) {
      this.status = PStatus.resolved
      this.res = res
      return MyP.resolve.bind(this)
    }
  }
  onReject(err: any) {
    if (this.status === PStatus.pending) {
      this.status = PStatus.rejected
      this.res = err
      return MyP.reject.bind(this)
    }
  }

  static resolve(res: any) {
    if (res instanceof MyP) {
      return res.run()
    } else {
      return new MyP((resolve, reject) => {
        resolve(res)
      }).run()
    }
  }
  static reject(err: any) {
    return new MyP((resolve, reject) => {
      reject(err)
    }).run()
  }

  static all(arr: Array<MyP>) {

  }
  static race(arr: Array<MyP>) {

  }

  then(onResolve: Function, onReject: Function) {
    let self = this

    if (this.status === PStatus.resolved) {
      return new MyP(function (resolve, reject) {
        var x = onResolve(self.res)
        if (x instanceof MyP) {
          x.then(resolve, reject)
        }
        resolve(x)
      })
    } else if (this.status === PStatus.rejected) {
      return new MyP(function (resolve, reject) {
        var x = onReject(self.res)
        if (x instanceof MyP) {
          x.then(resolve, reject)
        }
      })
    } else if (this.status === PStatus.pending) {
      return new MyP(function (resolve, reject) {
        self.cbArr.push(function (value) {
          try {
            var x = onResolve(self.res)
            if (x instanceof MyP) {
              x.then(resolve, reject)
            }
            resolve(x)
          } catch (e) {
            reject(e)
          }
        })

        self.errcbArr.push(function (reason) {
          try {
            var x = onReject(self.res)
            if (x instanceof MyP) {
              x.then(resolve, reject)
            }
          } catch (e) {
            reject(e)
          }
        })
      })
    }
  }
  catch(reject: Function) {
    return this.then(null, reject)
  }
  finally(onFinally: Function): MyP {
    return 
  }
}

export default MyP